Tools for visualizing and picking out trends from the text students submit in their assignments



In [ ]:
%cd ~/Dropbox/CanvasHacks

  
#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [ ]:
# %cd ../

from CanvasHacks import environment
from CanvasHacks.RequestTools import get_all_course_assignments, get_assignments_needing_grading
from CanvasHacks.Configuration import InteractiveConfiguration
from CanvasHacks.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
import CanvasHacks.GradingTools as GT
import CanvasHacks.DownloadProcessingTools as PT
from CanvasHacks.DataManagement import DataStore

from CanvasHacks.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.UploadGradeTools import make_upload_button

# This aren't used in the non-saving version
from CanvasHacks.FileTools import makeDataFileIterator
from CanvasHacks.JournalsFileTools import get_journal_folders, make_folder_list, calculate_journal_counts
from CanvasHacks.JournalsFileTools import journal_folder_name, create_folder


# Setup and credentials

# Choose assignments to analyze

In [ ]:
from IPython.display import display
from ipywidgets import widgets


# Get list of all assignments for the course
assignments = get_all_course_assignments(environment.CONFIG.course_ids[0])
assignments = [(a['id'], a['name']) for a in assignments]
# assignments

# Display a button for each
for assignment_id, name in assignments:
    b = widgets.Button( description=name )
    def callback( change ):
        environment.CONFIG.add_assignment(assignment_id, name)
    b.on_click( callback )
    display( b )

In [ ]:
environment.CONFIG.get_assignment_ids()

# Get and process student submissions

In [ ]:
name = 'j'
course_id = 2
b = widgets.Button( description='jjj' )

def callback( change ):
    environment.CONFIG.add_course_id(course_id)

b.on_click( callback )
display(b )



# Plot wordcount distributions

In [ ]:
START_WEEK = 2
STOP_WEEK = 15
DATA_FOLDER = '%s/data' % environment.PROJ_BASE
IGNORE_FILE = "%s/ignore.csv" % DATA_FOLDER

from CanvasTools.TextProcessing import make_wordbag
from CanvasTools.TextProcessing import WordFreq as WF

## Load data

In [ ]:
def week_key_gen(start=START_WEEK, stop=STOP_WEEK +1):
    """Create an iterator for all the keys to the various dictionaries"""
    for i in range(start, stop):
        yield "w%s" % i


In [ ]:
# Get all subfolder paths
from CanvasTools.JournalsFileTools import get_journal_folders, make_folder_list, calculate_journal_counts

# import os

# def get_journal_folders():
#     """Constructs paths to all journal folders and returns the list
#         example result: [
#             '/Users/adam/Box Sync/Phil 305 Business ethics/Student work/41181-Journal-week-2',
#              '/Users/adam/Box Sync/Phil 305 Business ethics/Student work/41181-Journal-week-3',
#              ....
#     """
#     journal_folders = []
#     for root, dirs, files in os.walk(environment.ARCHIVE_FOLDER):
#         for d in dirs:
#             journal_folders.append(os.path.join(root, d)) 
#     return journal_folders


# def make_folder_list(sections, week_num):
#     return ["%s-Journal-week-%s" % (s, week_num) for s in sections]


# # counts from each week
# def calculate_journal_counts(journal_folders):
#     """Returns the count of journals for each week"""
#     cnt = []
#     for r in journal_folders:
#         with open("%s/all-submissions.json" % r, 'r') as f:
#             j = json.load(f)
#         cnt.append((r.split('/')[-1:], len(j)))
#     return cnt

In [ ]:
from CanvasTools.JournalsFileTools import load_words_to_ignore

# def load_words_to_ignore(file=IGNORE_FILE):
#     words = []
#     with open(file, 'r') as csvfile:
#         reader = csv.DictReader(csvfile) #, delimiter=',', quotechar='|')
#         for row in reader:
# #             print (row['journals'])
#             words.append(row['journals'])
#     return words

In [ ]:
from CanvasTools.JournalsTextTools import filter_out_terms
# def filter_out_terms(combined):
#     """Loads the strings to ignore from IGNORE_FILE and returns the filtered wordbag"""
#     exclude = load_words_to_ignore()
# #     exclude = ['/p','style=', 'br', '“', 'ethics', '”', 'm','’','p', 'picture-uploaded', 'div', '/div',
# #               '305', 'september', '2018', 'text-align', 'class=', 'span', 'swenson', 'phil', 'philosophy', 
# #                '-webkit-standard', 'font', 'face='
# #               ]
#     exclude += [ "%s" % i for i in range(0,10)]
# #     print(exclude)

#     for k in combined.keys():
#         combined[k] = [w for w in combined[k] if w not in exclude]
#     return combined

In [ ]:
import csv
def write_csv( csvFile, rowsToWrite ):
    """TODO: let this figure out whether needs i, j or i"""
    with open( csvFile, 'a' ) as csvfile:
        writer = csv.writer( csvfile )
        # for i in toWrite:
        if type(rowsToWrite) is not list:
            rowsToWrite = [rowsToWrite]
        for r in rowsToWrite:
            print(r)
            writer.writerow(  r  )



In [ ]:

def clearplot_function( ):
    """Clears the matplotlib plot to free system resources. Call in next cell after plot"""
    try:
        plt.close( plt.gcf( ) ); plt.close( 'all' )
    except:
        pass



In [ ]:
exclude = ['/p','style=', 'br', '“', 'ethics', '”', 'm','’','p', 'picture-uploaded', 'div', '/div',
              '305', 'september', '2018', 'text-align', 'class=', 'span', 'swenson', 'phil', 'philosophy', 
               '-webkit-standard', 'font', 'face='
              ]

In [ ]:
# Create the file of terms to ignore
with open(IGNORE_FILE, 'w') as csvfile:
    fieldnames = ['journals', 'online_assignments']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for e in exclude:
        writer.writerow({'journals': e, 'online_assignments': ''})


In [ ]:
load_words_to_ignore()

In [ ]:
journal_folders = get_journal_folders()

In [ ]:
wcnt = calculate_journal_counts(journal_folders)
wcnt   

In [ ]:
# Load all content as wordbags
journal_folders = get_journal_folders()

data = {}
for r in journal_folders:
    with open("%s/all-submissions.json" % r, 'r') as f:
        j = json.load(f)
    data[r.split('/')[-1:][0]] = [make_wordbag(str(row['body'])) for row in j]

# data

In [ ]:
combined = {}
"""word bags containing all words from all courses for the week
has the format {'w2' : [], 'w3': [], 'w4' : []}"""


freqs = {}
"""contains a word frequency object for each week"""

w = START_WEEK

while w <= STOP_WEEK:
    folders = make_folder_list(sections, w)
    # add entry to the output dict
    key = 'w%s' % w
    combined[key] = []
    for k in folders:
        if k in data.keys():
            for b in data[k]:
                combined[key] += b
    w += 1

# Run filters on the combined wordbags
combined = filter_out_terms(combined)

# populate the dictionary with frequencies
for k in combined.keys():
    freqs[k] = WF(combined[k])



## Unigram visualizations

In [ ]:
# import wordcloud
from CanvasTools.VisualizationTools import draw_cloud, draw_cloud_from_freqs, draw_cumulative_freq

# def draw_cloud(wordlist, title, font_size=20):
#     """Given a list of words, it plots a wordcloud"""
#     # make into block of text
#     words = ' '.join(combined['w4'])

#     wc = wordcloud.WordCloud(
#                           width=2500,
#                           height=2000
#                          ).generate(words)
#     plt.figure(1,figsize=(13, 13))
#     plt.imshow(wc)
#     plt.axis('off')
#     plt.title(title)
#     ax = plt.gca()
#     ax.title.set_fontsize(font_size)
#     plt.show()

# def draw_cloud_from_freqs(freqs, week_num, title, font_size=20):
#     """Given the frequency object with keys like w1, w2,
#     this plots a wordcloud from those frequencies
#     """
#     f = {}
#     w = "w%s" % week_num
#     for k in freqs[w].freqDist:
#         f[k] = freqs[w].freqDist.freq(k)
  
#     wc = wordcloud.WordCloud(width=2500, height=2000).generate_from_frequencies(f)
#     plt.figure(1,figsize=(13, 13))
#     plt.imshow(wc)
#     plt.axis('off')
#     plt.title(title)
#     ax = plt.gca()
#     ax.title.set_fontsize(font_size)
#     plt.show()

# def draw_cumulative_freq(freqs, week_num, max_terms=30, font_size=20):
#     """Plots cumulative frequencies of terms"""
#     week = "w%s" % week_num

#     fig = plt.gcf()
#     ax = plt.gca()
#     ax.set_title(week)
    
# #     # rotate the x axis labels to 45 degrees
# #     for tick in ax.get_xticklabels():
# #         tick.set_rotation(45)


#     fig.set_figwidth(13)
#     fig.set_figheight(5)
#     # change the font size
#     for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +ax.get_xticklabels() + ax.get_yticklabels()):
#         item.set_fontsize(font_size)
    
#     plt.xticks(rotation=45)


#     freqs[week].plot(max_terms)
#     fig.tight_layout()


In [ ]:

# while True:
for i in range(START_WEEK, STOP_WEEK +1):
    # plot the cumulative frequencies
    draw_cumulative_freq(freqs, i)
    w = "w%s" % i
    # plot the wordcloud
    draw_cloud_from_freqs(freqs, i, 'Week %s Journals' % i)
#     draw_cloud(combined[w], 'week %s journals' % i)
    

## Ngram tools

In [ ]:
"""
Previously in TextProcessingTools
Created by adam on 11/11/15
"""
__author__ = 'adam'

import nltk

# This idiom is necessary. See https://github.com/nltk/nltk/issues/1516
from nltk.metrics import association



class NgramError(BaseException):
    def __init__(self, processing_step):
        """
        Arguments:
            :param processing_step: String description of where error arose
        :return:
        """
        super().__init__()
        self.kind = 'NgramProcessing'
        self.identifier_type = 'String content'
        self.step = processing_step
#         ProcessingError.__init__(self, processing_step)

class NgramGetter(object):
    """
    Abstract parent class for extracting ngrams.

    Attributes:
        collocation_finder: One of the nltk's collocation finder tools (e.g., BigramCollocationFinder)
        top_likelihood_ratio:
        measurement_tool: One of nltk's measurement tools (e.g., nltk.collocations.BigramAssocMeasures)
        modifiers: IModifier instantiating tool for modifying the text before calculating ngrams
        ngrams: List of ngrams
        raw_freq: Frequency distribution of ngrams
        sorted_ngrams: List of tuples sorted by self.scored_ngrams
        top_pmi: Variable number of n-grams with the highest Pointwise Mutual Information (i.e., which occur together
        more often than would be expected)
        word_bag: List of text to run
    """

    def __init__(self):
        self.modifiers = []
        self.ngram_filters = []
        self.word_bag = []
        self.ngrams = []
        if not self.measurement_tool:
            raise NotImplementedError

    def add_modifier(self, iModifier):
        assert(isinstance(iModifier, IModifier))
        self.modifiers.append(iModifier)

    def _run_modifiers(self):
        """
        Calls the modifiers in sequence and stores the results back in word_bag
        """
        for modifier in self.modifiers:
            self.word_bag = [modifier.process(w) for w in self.word_bag]

    def add_filter(self, iNgramFilter):
        """
        Adds a filter to be run after the ngrams are created
        :param iNgramFilter:
        :return:
        """
        self.ngram_filters.append(iNgramFilter)

    def apply_filters(self):
        for ftr in self.ngram_filters:
            self.collocation_finder.apply_ngram_filter(ftr)

    def process(self, word_bag, min_freq=3, get_top=10, **kwargs):
        """
        Runs any modifiers (stemmers, lemmatizers, etc) on the list of terms and
        then extracts the ngrams

        Args:
            get_top: The cut off for ngrams to get stats for
            min_freq: Integer of minimum number of appearances of ngram to extract
            word_bag: List of strings to extract ngrams from. Should already be filtered.
        """
        raise NotImplementedError

    def _calculate_statistics(self, get_top=10, **kwargs):
        """
                A number of measures are available to score collocations or other associations.
        The arguments to measure functions are marginals of a contingency table,
        in the bigram case (n_ii, (n_ix, n_xi), n_xx):
                w1    ~w1
             ------ ------
         w2 | n_ii | n_oi | = n_xi
             ------ ------
        ~w2 | n_io | n_oo |
             ------ ------
             = n_ix        TOTAL = n_xx
        We test their calculation using some known values presented
        in Manning and Schutze's text and other papers.
        Student's t: examples from Manning and Schutze 5.3.2
        Arguments:
            get_top: The cut off for ngrams to get stats for
        """
        self.topPMI = self.collocation_finder.nbest(self.measurement_tool.pmi, get_top)
        self.raw_freq = self.collocation_finder.score_ngrams(self.measurement_tool.raw_freq)
        self.sorted_ngrams = [ngram for ngram, score in self.raw_freq]
        self.top_likelihood_ratio = self.collocation_finder.nbest(self.measurement_tool.likelihood_ratio, get_top)


class BigramGetter(NgramGetter):
    """
    Extracts 2-grams from a word bag and calculates statistics
    Attributes:
        top_pmi: Variable number of n-grams with the highest Pointwise Mutual Information (i.e., which occur together
        more often than would be expected)
        top_likelihood_ratio:
        raw_freq: Frequency distribution of ngrams
        sorted_ngrams: List of tuples sorted by self.scored_ngrams
    """

    def __init__(self):
        self.measurement_tool = association.BigramAssocMeasures()
        NgramGetter.__init__(self)

    def process(self, word_bag, min_freq=3, get_top=10, **kwargs):
        """
        Arguments:
            word_bag: List of strings
        """
        assert(isinstance(word_bag, list))
        self.collocation_finder = nltk.collocations.BigramCollocationFinder.from_words(word_bag)
        self.collocation_finder.apply_freq_filter(min_freq)
        self._calculate_statistics(get_top)

class TrigramGetter(NgramGetter):
    """
        Extracts 3-grams from a word bag and calculates statistics
    """

    def __init__(self):
        self.measurement_tool = association.TrigramAssocMeasures()
        NgramGetter.__init__(self)

    def process(self, word_bag, min_freq=3, get_top=10, **kwargs):
        """
        Arguments:
            word_bag: List of strings
        """
        assert(isinstance(word_bag, list))
#         try:
        self._run_modifiers()
        self.collocation_finder = nltk.collocations.TrigramCollocationFinder.from_words(word_bag)
        self.collocation_finder.apply_freq_filter(min_freq)
        self._calculate_statistics(get_top)


### Find bigrams

In [ ]:
from nltk.probability import FreqDist

bigrams = {}
week_iter = week_key_gen()
i = START_WEEK

while True:
    try:
        w = next(week_iter)
        
        bg = BigramGetter()
        bg.process(combined[w])
        #store the bigram object in our dict
        bigrams[w] = bg

        # reshape into a usable form for plotting
        fs = FreqDist()
        for s, freq in bg.raw_freq:
            # make the string key and set the frequency
            fs["%s %s" % (s[0], s[1])] = freq

        # plot the cumulative frequencies
        draw_cumulative_freq({w : fs}, i)
        
        i += 1
    except StopIteration:
        break

In [ ]:
clearplot_function()

### Likelihood ratio

In [ ]:
# Print top likelihood ratio
def print_top_likelihood_ratios(ngrams):
    week_iter = week_key_gen()

    while True:
        try:
            week = next(week_iter)
            print(week)
            [print(b) for b in ngrams[week].top_likelihood_ratio]
        except StopIteration:
            break

def print_top_PMI(ngrams):
    week_iter = week_key_gen()
    print("Top PMI")

    while True:
        try:
            week = next(week_iter)
            print(week)
            [print(b) for b in ngrams[week].topPMI]
        except StopIteration:
            break


In [ ]:
print_top_likelihood_ratios(bigrams)

In [ ]:
print_top_PMI(bigrams)

## Find trigrams

In [ ]:
trigrams = {}
week_iter = week_key_gen()
i = START_WEEK

while True:
    try:
        w = next(week_iter)
        
        bg = TrigramGetter()
        bg.process(combined[w])
        #store the trigram object in our dict
        trigrams[w] = bg

        # reshape into a usable form for plotting
        fs = FreqDist()
        for s, freq in bg.raw_freq:
            # make the string key and set the frequency
            fs["%s %s %s" % (s[0], s[1], s[2])] = freq

        # plot the cumulative frequencies
        draw_cumulative_freq({w : fs}, i)
        i += 1
    except StopIteration:
        break

In [ ]:
clearplot_function()

In [ ]:
print_top_likelihood_ratios(trigrams)

In [ ]:
print_top_PMI(trigrams)